# Phase 3.3: Loading Models from the Registry

This comprehensive notebook demonstrates:
1. **Load by Version Number** - Get a specific model version
2. **Load by Stage Name** - Get the model in Production/Staging
3. **Get Model Metadata** - Access version info and metrics
4. **Best Practices** - Production model loading patterns

## Prerequisites
**Important:** Run `02_model_stages.ipynb` first to create the models!

## Why Load from Registry?

Loading from the registry instead of direct paths provides:
- **Version Control**: Load specific versions by number
- **Stage-Based Loading**: Always get the "Production" model
- **Consistency**: Same URI pattern across environments
- **Abstraction**: Don't need to know file paths

## Model URI Formats

| Format | Example | Use Case |
|--------|---------|----------|
| By Version | `models:/my-model/1` | Load specific version |
| By Stage | `models:/my-model/Production` | Load production model |
| By Run | `runs:/<run_id>/model` | Load from specific run |

## Learning Goals
- Load models by version number
- Load models by stage name
- Access model metadata and metrics
- Use best practices for production loading

## Step 1: Import Libraries

In [ ]:
# MLflow imports
import mlflow
import mlflow.pyfunc
from mlflow.tracking import MlflowClient

# sklearn for test data
from sklearn.datasets import load_iris

# Data handling
import pandas as pd
import os

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")
print("Ready to learn about loading models from registry!")

## Step 2: Connect to MLflow

In [ ]:
# Get MLflow tracking server URL
TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI", "http://localhost:5000")

# Connect to MLflow
mlflow.set_tracking_uri(TRACKING_URI)

# Model we created in the previous notebook
MODEL_NAME = "iris-classifier-staged"

# Create client for registry operations
client = MlflowClient()

print(f"Connected to MLflow at: {TRACKING_URI}")
print(f"Looking for model: {MODEL_NAME}")

## Step 3: Check if Model Exists

In [ ]:
# Verify the model exists in the registry
try:
    model_info = client.get_registered_model(MODEL_NAME)
    print(f"Model found: {model_info.name}")
    print(f"Description: {model_info.description or '(no description)'}")
except Exception as e:
    print(f"ERROR: Model '{MODEL_NAME}' not found!")
    print("Please run 02_model_stages.ipynb first.")
    print(f"\nError details: {e}")

## Step 4: Prepare Test Data

In [ ]:
# Load test data for predictions
iris = load_iris()

# Create a small test set (first 5 samples)
X_test = pd.DataFrame(iris.data[:5], columns=iris.feature_names)
y_test = iris.target[:5]

print("Test Data (5 samples):")
print("="*60)
print(X_test.to_string(index=False))
print(f"\nActual labels: {list(y_test)}")
print(f"Class names: {[iris.target_names[i] for i in y_test]}")

## Method 1: Load by Version Number

Use this when you need a **specific version** of the model.

**URI Format:** `models:/<model_name>/<version>`

In [ ]:
print("="*60)
print("[Method 1: Load by Version Number]")
print("="*60)

# Try loading each version
for version in ["1", "2", "3"]:
    # Construct the model URI
    # Format: models:/<model_name>/<version_number>
    model_uri = f"models:/{MODEL_NAME}/{version}"
    
    try:
        # Load the model
        model = mlflow.pyfunc.load_model(model_uri)
        
        # Make predictions
        predictions = model.predict(X_test)
        
        print(f"\nVersion {version}:")
        print(f"  URI: {model_uri}")
        print(f"  Predictions: {list(predictions)}")
        print(f"  Classes: {[iris.target_names[p] for p in predictions]}")
        
    except Exception as e:
        print(f"\nVersion {version}: Failed to load")
        print(f"  Error: {e}")

## Method 2: Load by Stage Name

Use this when you want the model at a **specific stage** (e.g., always get Production).

**URI Format:** `models:/<model_name>/<stage>`

This is especially useful in deployment scripts where you always want the current production model.

In [ ]:
print("\n" + "="*60)
print("[Method 2: Load by Stage Name]")
print("="*60)

# Available stages to try
stages = ["None", "Staging", "Production", "Archived"]

for stage in stages:
    # Construct the model URI with stage
    # Format: models:/<model_name>/<stage>
    model_uri = f"models:/{MODEL_NAME}/{stage}"
    
    try:
        # Load the model
        model = mlflow.pyfunc.load_model(model_uri)
        
        # Make predictions
        predictions = model.predict(X_test)
        
        print(f"\n{stage}:")
        print(f"  URI: {model_uri}")
        print(f"  Loaded successfully!")
        print(f"  Predictions: {list(predictions)}")
        
    except Exception as e:
        print(f"\n{stage}:")
        print(f"  No model in this stage (or error: {str(e)[:50]}...)")

## Method 3: Get Latest Versions by Stage

Use the MlflowClient to get information about the latest version in each stage before loading.

In [ ]:
print("\n" + "="*60)
print("[Method 3: Get Latest Versions by Stage]")
print("="*60)

for stage in ["Staging", "Production"]:
    # Get the latest version(s) in this stage
    versions = client.get_latest_versions(MODEL_NAME, stages=[stage])
    
    if versions:
        latest = versions[0]  # Get the first (latest) one
        
        print(f"\nLatest in {stage}:")
        print(f"  Version: {latest.version}")
        print(f"  Run ID: {latest.run_id[:8]}...")
        print(f"  Status: {latest.status}")
        
        # Load and predict
        model = mlflow.pyfunc.load_model(f"models:/{MODEL_NAME}/{latest.version}")
        predictions = model.predict(X_test)
        print(f"  Predictions: {list(predictions)}")
    else:
        print(f"\nNo versions in {stage}")

## Method 4: Get Detailed Model Metadata

Access comprehensive information about each model version, including the metrics from the training run.

In [ ]:
print("\n" + "="*60)
print("[Method 4: Model Metadata]")
print("="*60)

# Search for all versions of our model
for mv in client.search_model_versions(f"name='{MODEL_NAME}'"):
    print(f"\nVersion {mv.version}:")
    print(f"  Stage: {mv.current_stage}")
    print(f"  Status: {mv.status}")
    print(f"  Run ID: {mv.run_id}")
    print(f"  Source: {mv.source[:50]}...")
    print(f"  Created: {mv.creation_timestamp}")
    
    # Get the run to access metrics
    run = client.get_run(mv.run_id)
    accuracy = run.data.metrics.get("accuracy", "N/A")
    print(f"  Accuracy: {accuracy:.4f}" if isinstance(accuracy, float) else f"  Accuracy: {accuracy}")
    
    # Get parameters
    params = run.data.params
    if params:
        print(f"  Parameters: {params}")

## Best Practice: Production Model Loading

Here's the recommended pattern for loading the production model in your application.

In [ ]:
print("\n" + "="*60)
print("[Best Practice: Production Model Loading]")
print("="*60)

try:
    # This is how you'd load in production code:
    # Always use the stage name, not version number
    # This way, when you promote a new model, your code automatically uses it
    
    production_model = mlflow.pyfunc.load_model(
        f"models:/{MODEL_NAME}/Production"
    )
    
    print("\nProduction model loaded successfully!")
    print(f"Model type: {type(production_model).__name__}")
    
    # Make predictions
    print("\nMaking predictions on test data...")
    predictions = production_model.predict(X_test)
    class_names = [iris.target_names[p] for p in predictions]
    
    # Display results
    print("\nResults:")
    print("-" * 55)
    print(f"{'#':<4} {'Predicted':<15} {'Actual':<15} {'Match'}")
    print("-" * 55)
    
    for i, (pred_name, actual_idx) in enumerate(zip(class_names, y_test), 1):
        actual_name = iris.target_names[actual_idx]
        match = "correct" if pred_name == actual_name else "wrong"
        print(f"{i:<4} {pred_name:<15} {actual_name:<15} {match}")
    
    print("-" * 55)
    accuracy = (predictions == y_test).mean()
    print(f"\nAccuracy on test samples: {accuracy:.1%}")
    
except Exception as e:
    print(f"\nFailed to load production model: {e}")
    print("\nMake sure you have a model in the 'Production' stage.")
    print("Run 02_model_stages.ipynb to set this up.")

## Summary: Loading from Registry

### URI Formats

```python
# By version number (specific version)
model = mlflow.pyfunc.load_model("models:/my-model/1")

# By stage (recommended for production)
model = mlflow.pyfunc.load_model("models:/my-model/Production")

# By run ID (from experiments)
model = mlflow.pyfunc.load_model("runs:/<run_id>/model")
```

### Best Practices

| Scenario | Recommended Approach |
|----------|---------------------|
| Production serving | Load by stage: `models:/name/Production` |
| Testing specific version | Load by version: `models:/name/3` |
| Debugging a run | Load by run: `runs:/<run_id>/model` |
| Staging tests | Load by stage: `models:/name/Staging` |

### Key Advantage of Stage-Based Loading

```python
# Your production code stays the same:
model = mlflow.pyfunc.load_model("models:/my-model/Production")

# When you promote version 5 to Production,
# your code automatically uses it - no code changes needed!
```

In [ ]:
print("="*60)
print("Loading from Registry Tutorial Complete!")
print("="*60)
print(f"\nView at: {TRACKING_URI}")
print("\nWhat you learned:")
print("  1. How to load models by version number")
print("  2. How to load models by stage name")
print("  3. How to get model metadata and metrics")
print("  4. Best practices for production model loading")
print("  5. Why stage-based loading is recommended")